In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Crohns_Disease"
cohort = "GSE123088"

# Input paths
in_trait_dir = "../../input/GEO/Crohns_Disease"
in_cohort_dir = "../../input/GEO/Crohns_Disease/GSE123088"

# Output paths
out_data_file = "../../output/preprocess/Crohns_Disease/GSE123088.csv"
out_gene_data_file = "../../output/preprocess/Crohns_Disease/gene_data/GSE123088.csv"
out_clinical_data_file = "../../output/preprocess/Crohns_Disease/clinical_data/GSE123088.csv"
json_path = "../../output/preprocess/Crohns_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: CD4+ T cells'], 1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'], 2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset likely contains gene expression data, so:
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait: Locate Crohn's Disease data in sample characteristics dictionary
trait_row = 1  # Primary diagnosis is in row 1, which includes "CROHN_DISEASE"

# Age: Age information is available in row 3 and 4
age_row = 3  # Using row 3 as the main source for age data

# Gender: Gender information is in row 2 and 3 (labeled as "Sex")
gender_row = 2  # Using row 2 as the main source for gender data

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait values to binary (1 for Crohn's Disease, 0 for Control)"""
    if isinstance(value, str):
        value = value.strip().lower()
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'crohn' in value or 'crohn_disease' in value:
            return 1
        elif 'control' in value or 'healthy_control' in value:
            return 0
    return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if isinstance(value, str):
        value = value.strip()
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        try:
            return float(value)
        except ValueError:
            pass
    return None

def convert_gender(value):
    """Convert gender values to binary (0 for Female, 1 for Male)"""
    if isinstance(value, str):
        value = value.strip().lower()
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'female' in value:
            return 0
        elif 'male' in value:
            return 1
    return None

# 3. Save Metadata
# Initial filtering on usability
is_trait_available = trait_row is not None
initial_validation = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_selected = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_selected)
    print("Preview of clinical data:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_selected.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM3494884': [nan, 56.0, 1.0], 'GSM3494885': [nan, nan, nan], 'GSM3494886': [nan, 20.0, 0.0], 'GSM3494887': [nan, 51.0, 0.0], 'GSM3494888': [nan, 37.0, 1.0], 'GSM3494889': [nan, 61.0, 1.0], 'GSM3494890': [nan, nan, nan], 'GSM3494891': [nan, 31.0, 1.0], 'GSM3494892': [nan, 56.0, 0.0], 'GSM3494893': [nan, 41.0, 0.0], 'GSM3494894': [nan, 61.0, 0.0], 'GSM3494895': [nan, nan, nan], 'GSM3494896': [nan, 80.0, 1.0], 'GSM3494897': [nan, 53.0, 1.0], 'GSM3494898': [nan, 61.0, 1.0], 'GSM3494899': [nan, 73.0, 1.0], 'GSM3494900': [nan, 60.0, 1.0], 'GSM3494901': [nan, 76.0, 1.0], 'GSM3494902': [nan, 77.0, 0.0], 'GSM3494903': [nan, 74.0, 0.0], 'GSM3494904': [nan, 69.0, 1.0], 'GSM3494905': [nan, 77.0, 0.0], 'GSM3494906': [nan, 81.0, 0.0], 'GSM3494907': [nan, 70.0, 0.0], 'GSM3494908': [nan, 82.0, 0.0], 'GSM3494909': [nan, 69.0, 0.0], 'GSM3494910': [nan, 82.0, 0.0], 'GSM3494911': [nan, 67.0, 0.0], 'GSM3494912': [nan, 67.0, 0.0], 'GSM3494913': [nan, 78.0, 0.0], 'GSM3494914': [n

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')

Gene data dimensions: 24166 genes × 204 samples


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# These appear to be integer/numeric identifiers rather than standard HGNC gene symbols
# Human gene symbols are typically alphanumeric (like "TP53", "BRCA1", "IL6")
# The identifiers shown are simple numbers which likely need mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# 3. Preview the gene annotation dataframe
print("Gene annotation dataframe column names:")
print(gene_annotation.columns)

# Preview the first few rows to understand the data structure
print("\nPreview of gene annotation data:")
annotation_preview = preview_df(gene_annotation, n=3)
print(annotation_preview)

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Gene annotation dataframe column names:
Index(['ID', 'ENTREZ_GENE_ID', 'SPOT_ID'], dtype='object')

Preview of gene annotation data:
{'ID': ['1', '2', '3'], 'ENTREZ_GENE_ID': ['1', '2', '3'], 'SPOT_ID': [1.0, 2.0, 3.0]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene identifiers in both datasets
# From previous outputs:
# - Gene expression data: identifiers are numeric strings like '1', '2', '3'
# - Gene annotation data: has columns 'ID', 'ENTREZ_GENE_ID', 'SPOT_ID'

# The 'ID' column in gene annotation matches the identifiers in gene expression data
# The 'ENTREZ_GENE_ID' column appears to contain Entrez Gene IDs which we need to map to gene symbols

# 2. Create gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='ENTREZ_GENE_ID')

# Print information about the mapping
print(f"Number of probes with mappings: {len(mapping_df)}")
print(f"Sample of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the mapped gene data
print("\nAfter mapping, gene data dimensions:", gene_data.shape)
print("First 10 mapped gene symbols (if any):")
if len(gene_data) > 0:
    print(gene_data.index[:10])
else:
    print("No genes were successfully mapped. This might indicate an issue with the gene annotation format.")
    
    # Save original probe data as gene data for downstream processes
    # This is a fallback to allow the pipeline to continue
    gene_data = get_genetic_data(matrix_file)
    gene_data.index.name = 'Gene'
    print(f"\nUsing original probe data: {gene_data.shape[0]} probes × {gene_data.shape[1]} samples")


Number of probes with mappings: 4740924
Sample of mapping data:
   ID Gene
0   1    1
1   2    2
2   3    3
3   9    9
4  10   10



After mapping, gene data dimensions: (0, 204)
First 10 mapped gene symbols (if any):
No genes were successfully mapped. This might indicate an issue with the gene annotation format.



Using original probe data: 24166 probes × 204 samples


### Step 7: Data Normalization and Linking

In [8]:
# 1. Make sure we have the original gene_data before any mapping attempts
original_gene_data = get_genetic_data(matrix_file)
print(f"Original probe data dimensions: {original_gene_data.shape}")

# Handle the case where gene data is empty after mapping
if gene_data.shape[0] == 0:
    print("Warning: No genes were successfully mapped to symbols. Using probe IDs as features instead.")
    # Use the original probe-level data instead of the empty mapped data
    gene_data_normalized = original_gene_data
    print(f"Using original probe data as features: {gene_data_normalized.shape[0]} probes × {gene_data_normalized.shape[1]} samples")
else:
    # If we have mapped gene data, proceed with normalization
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

# Save the gene data (normalized or probe-based)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Load clinical data
print("Loading clinical features...")
# Check if the clinical data file exists
if os.path.exists(out_clinical_data_file):
    print(f"Loading clinical features from {out_clinical_data_file}")
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
else:
    # Extract clinical features from the original source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_raw = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df=clinical_raw,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_data.to_csv(out_clinical_data_file)

print("Clinical data preview:")
print(preview_df(clinical_data))
print(f"Clinical data shape: {clinical_data.shape}")

# Link clinical and genetic data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(clinical_data, gene_data_normalized)
print(f"Linked data shape: {linked_data.shape}")

# Verify we have both trait and gene data in the linked dataset
has_trait_data = trait in linked_data.columns
has_genetic_data = len(linked_data.columns) > 3  # More than just trait, age, and gender columns

if not has_trait_data or not has_genetic_data:
    print("Error: Linked data is missing trait data or gene expression data.")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=has_genetic_data,
        is_trait_available=has_trait_data,
        is_biased=True,
        df=linked_data,
        note="Failed to properly link gene expression data with clinical features."
    )
    print("Dataset deemed not usable due to incomplete data.")
else:
    # Handle missing values
    print("Handling missing values...")
    linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
    print(f"Data shape after handling missing values: {linked_data_clean.shape}")
    
    # If we have samples after missing value handling
    if linked_data_clean.shape[0] > 0:
        # Check for bias in features
        print("\nChecking for bias in feature variables:")
        is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
        
        # Final quality validation
        note = "Dataset contains gene expression data (using probe IDs) for Crohn's Disease patients."
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=is_biased,
            df=linked_data_final,
            note=note
        )
        
        # Save linked data if usable
        if is_usable:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            linked_data_final.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
            print(f"Final dataset shape: {linked_data_final.shape}")
        else:
            print("Dataset deemed not usable for trait association studies, linked data not saved.")
    else:
        print("Error: No samples remain after handling missing values.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=pd.DataFrame(),
            note="All samples were removed during missing value handling."
        )
        print("Dataset deemed not usable as all samples were filtered out.")

Original probe data dimensions: (24166, 204)
Normalizing gene symbols...
Gene data shape after normalization: 0 genes × 204 samples
Gene expression data saved to ../../output/preprocess/Crohns_Disease/gene_data/GSE123088.csv
Loading clinical features...
Loading clinical features from ../../output/preprocess/Crohns_Disease/clinical_data/GSE123088.csv
Clinical data preview:
{'GSM3494884': [nan, 56.0, 1.0], 'GSM3494885': [nan, nan, nan], 'GSM3494886': [nan, 20.0, 0.0], 'GSM3494887': [nan, 51.0, 0.0], 'GSM3494888': [nan, 37.0, 1.0], 'GSM3494889': [nan, 61.0, 1.0], 'GSM3494890': [nan, nan, nan], 'GSM3494891': [nan, 31.0, 1.0], 'GSM3494892': [nan, 56.0, 0.0], 'GSM3494893': [nan, 41.0, 0.0], 'GSM3494894': [nan, 61.0, 0.0], 'GSM3494895': [nan, nan, nan], 'GSM3494896': [nan, 80.0, 1.0], 'GSM3494897': [nan, 53.0, 1.0], 'GSM3494898': [nan, 61.0, 1.0], 'GSM3494899': [nan, 73.0, 1.0], 'GSM3494900': [nan, 60.0, 1.0], 'GSM3494901': [nan, 76.0, 1.0], 'GSM3494902': [nan, 77.0, 0.0], 'GSM3494903': [nan,